In [1]:
import requests
from sklearn.manifold import TSNE
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd


In [27]:
#loading block
mykey = open('spoonac/apikey.txt').read().strip()
w2vm = pickle.load(open("bigoven/w2vmodel.pkl", 'rb'))
aisledict = pickle.load(open("bigoven/aisleclassification.pkl", 'rb'))
noise = pickle.load(open("bigoven/noiselist.pkl", 'rb'))
atFM = pickle.load(open("bigoven/FMproducts.pkl", 'rb'))



In [3]:
#from the user
userinput = 'https://www.simplyrecipes.com/recipes/chicken_piccata/'
params = {'url': userinput, 'forceExtraction': 'true', 'apiKey': mykey, 'analyze': 'true'}

In [4]:
#get infor about recipe they requested
response = requests.get('https://api.spoonacular.com/recipes/extract', params=params)


In [37]:
#extract ingredients from their recipe
rec = response.json()
ingcomp = rec['extendedIngredients']
ingredients = [','.join([ing['name'].lower() for ing in ingcomp ])] #add the stripping



#print(repr(ingredients)) # repr is the representation 
ingredients

['chicken breast halves,flour,salt,ground pepper,parmesan cheese,extra virgin olive oil,butter,chicken stock,lemon juice,brined capers,parsley']

In [41]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 

def removenoise(input_ing):
    noise_free_ing = []
    for word in input_ing.split(','):
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in noise)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        
#stem? lemmatize? part of speech tagging? stop words? synonym finding (https://wordnet.princeton.edu/)? pantry?
    

In [48]:
#call the dedupe functions
nning = []
for i in ingredients:
    nning.append(removenoise(i))



In [49]:
nning

[['chicken breast halves',
  'flour',
  'salt',
  'pepper',
  'parmesan cheese',
  'extra virgin olive oil',
  'butter',
  'chicken stock',
  'lemon juice',
  'brined capers',
  'parsley']]

In [110]:
#go through ingredients, if theyre not available at the FM use w2v and aisle to find similar 
available = []
toreplace = []
replacement = []
for i in nning[0]:
    if i in atFM:
        available.append(i)
    else:    
        toreplace.append(i)
        curaisle = aisledict[i]
        similar = w2vm.wv.most_similar(i, topn=1)
        opposite = w2vm.wv.most_similar(similar, topn=100)
        for opp in opposite:
            if opp[0] != i:
               # print(opp[0] , i)
                oppaisle = aisledict[opp[0]]
                if oppaisle == curaisle:
                    replacement.append(opp)
                    break
        print('Replacing ' + i + ' with ' + opp[0])    
            
            




Replacing chicken breast halves with slab bacon
Replacing salt with mustard powder
Replacing pepper with onion salt
Replacing parmesan cheese with fontina cheese
Replacing extra virgin olive oil with extravirgin olive oil
Replacing chicken stock with turkey stock
Replacing lemon juice with raspberries
Replacing brined capers with chanterelle mushrooms


In [117]:
aisledict['parsley']

['Produce', 'Spices and Seasonings']